In [85]:
# Install the split-folders to split the dataset into train, validation and test
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import zipfile
import splitfolders

In [2]:
# Download dataset from Google Drive
! gdown --id 1yAx9epfEO0SebwQ4MMZhnh1xRkalXhws

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1yAx9epfEO0SebwQ4MMZhnh1xRkalXhws
To: /content/kagglecatsanddogs_5340.zip
100% 825M/825M [00:05<00:00, 147MB/s]


In [3]:
# Open the zip file and create a new folder as Dataset
zip_file_local = '/content/kagglecatsanddogs_5340.zip'
zip_file_reference = zipfile.ZipFile(zip_file_local, 'r')
zip_file_reference.extractall('/content/Dataset')
zip_file_reference.close()

In [88]:
input_folder = "/content/Dataset/PetImages"
# Split dataset into train, validation and test dataset with a ratio of 0.7/0.2/0.1
splitfolders.ratio(input_folder, output="/content/Dataset/splitted_dataset",
                   seed=42, ratio=(0.7, 0.2, 0.1),
                   group_prefix=None)

Copying files: 25002 files [01:13, 338.13 files/s] 


In [89]:
dataset_direction = "/content/Dataset/PetImages"
train_direction = "/content/Dataset/splitted_dataset/train"
validation_direction = "/content/Dataset/splitted_dataset/val"

In [90]:
image_height, image_width=224,224
batch_size=32
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  train_direction,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

Found 17498 files belonging to 2 classes.
Using 13999 files for training.


In [91]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  validation_direction,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

Found 5000 files belonging to 2 classes.
Using 1000 files for validation.


In [92]:
# Initialize ResNet50
ResNet50_model = Sequential()
pretrained_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
)

# Set the first layer of the model to be not trainable, because ResNet-50 model was already trained.
for layer in pretrained_model.layers:
    layer.trainable = False

In [93]:
# Summary of the Pretrained Model
pretrained_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [22]:
#ResNet50_model.add(pretrained_model)

In [97]:
last_layer = pretrained_model.get_layer('conv5_block3_out')
last_output = last_layer.output
x = keras.layers.Flatten()(last_output)
x = keras.layers.Dense(64, activation='relu', name='FC_1')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(2, activation='softmax', name='sigmoid')(x)

In [98]:
resnet50_model = keras.Model(inputs = pretrained_model.input, outputs = x) 

In [99]:
# Summary of the ResNet50 Model
resnet50_model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [100]:
resnet50_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [62]:
print(train_dataset)
print(validation_dataset)
print(resnet50_model)

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [101]:
train_images = keras.preprocessing.image.ImageDataGenerator().flow_from_directory(train_direction,
                        target_size=(224,224),
                        batch_size=10)
test_images = keras.preprocessing.image.ImageDataGenerator().flow_from_directory(validation_direction,
                        target_size=(224,224),
                        batch_size=30)


Found 17498 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
history = resnet50_model.fit(train_images, validation_data=test_images, epochs=1, steps_per_epoch=10)

10/10 [==============================] - ETA: 0s - loss: 0.2102 - accuracy: 0.9000